# **STUDENT NAMES (أسماء الطلاب) :** <br>
# (ahmad_187728) أحمد محمد الشيخ خليل <br>
# (jana_198418) جنا ناصيف      
# (Lina_131051) لينه مرهف ابوحرب <br>  

In [1]:
import heapq as hq # Heap used to represent priority queue
import copy
from time import perf_counter

def calc_manhattan_dist(state, goal):
    all_dist = []
    for i in range(0, len(state)):
        for j in range(0, len(state[i])):
            if (state[i][j] == goal[i][j]):
                continue
            if (state[i][j] == 0):
                continue
            else:
                # find where this tile is in the goal state
                i_of_goal_tile, j_of_goal_tile = find_in_sublists(state[i][j], goal)

                distance = abs(i - i_of_goal_tile) + abs(j - j_of_goal_tile)
                all_dist.append(distance)
    return sum(all_dist)

def calc_misplaced_tiles(state, goal):
    """Calculate the number of misplaced tiles. 0 is ignored in this calculation."""
    # Flatten the list of lists for easy comparison
    state = [item for sublist in state for item in sublist]
    goal = [item for sublist in goal for item in sublist]
    count = 0
    for i in range(0, len(goal)):
        if state[i] == 0:
            continue
        if (state[i] != goal[i]):
            count += 1
    return count


def check_states_equal(state_a, state_b):
    """
    Check if two states are equal.
    Returns:
    true/false if states are the same/diff.
    """
    # Flatten the list of lists for easy comparison
    flat_list_a = [item for sublist in state_a for item in sublist]
    flat_list_b = [item for sublist in state_b for item in sublist]
    return flat_list_a == flat_list_b

def find_in_sublists(val, lst):
    """
    Helper function used in Node.expand. Find value in list of lists.
    Returns:
    A tuple containing the indices of val in lst.
    """
    for i, sub_list in enumerate(lst):
        try:
            j = sub_list.index(val)
        except ValueError:
            continue
        return i, j

    return None, None
def print_mystate(state):
    for row in state:
        print(' '.join(map(str,row)))

class Node:
    def __init__(self, parent=None, state=0, cost_from_start=0, cost_to_goal=0):
        self.parent = parent
        self.state = state
        self.cost_from_start = cost_from_start
        self.cost_to_goal = cost_to_goal
        self.children = []

    def __eq__(self, other):
        return self.state == other.state

    def __lt__(self, other):
        """This will automatically calculate the f-val = g(n) + h(n) when this node is put in a heapq"""
        return (self.cost_from_start + self.cost_to_goal) < (other.cost_from_start + other.cost_to_goal)

    def add_child(self, node, cost_to_expand=1):
        """Adds a child node to the list of this node's children"""
        node.cost_from_start = self.cost_from_start + cost_to_expand # extend the child'd cost from start
        node.parent = self # set parent
        self.children.append(node)

    def get_f_val(self):
        """Returns cost from start + cost to goal"""
        return (self.cost_from_start + self.cost_to_goal)

    def traceback(self):
        """Traverse all parents and print their states and traversed count"""
        print('Moves to solve:')
        print_mystate(self.state)

        p = self.parent
        count = -1
        while p:
            count += 1
            print()
            print('Move ',count)
            print()
            print_mystate(p.state)
            p = p.parent
        return count


    def t_count(self):
        """Traverse all parents and print their states and traversed count"""
        p = self.parent
        count = -1
        while p:
            count += 1
            p = p.parent
        return count+1

    def print_state(self):
        for i in self.state:
            print(i)

    def expand(self):
        """Returns a list of valid expanded states."""
        # find index of 0
        i_zero, j_zero = find_in_sublists(0, self.state)
        state_len = len(self.state)

        states_to_return = []

        # Move Up: Swap zero and the element above it such element exists
        if (i_zero != 0):
            tmp_state = copy.deepcopy(self.state)
            tmp_state[i_zero][j_zero] = tmp_state[i_zero - 1][j_zero]
            tmp_state[i_zero - 1][j_zero] = 0

            # Only add if the parent does not have the same expanded state
            if self.parent and check_states_equal(tmp_state, self.parent.state):
                states_to_return.append(None)
            else:
                states_to_return.append(tmp_state)

        # Move Right: Swap zero and the element on its right if possible
        if (j_zero != (state_len - 1)):
            tmp_state = copy.deepcopy(self.state)
            tmp_state[i_zero][j_zero] = tmp_state[i_zero][j_zero + 1]
            tmp_state[i_zero][j_zero + 1] = 0

            # Only add if the parent does not have the same expanded state
            if self.parent and check_states_equal(tmp_state, self.parent.state):
                states_to_return.append(None)
            else:
                states_to_return.append(tmp_state)

        # Move Down: Swap zero and the element below it if possible
        if (i_zero != (state_len - 1)):
            tmp_state = copy.deepcopy(self.state)
            tmp_state[i_zero][j_zero] = tmp_state[i_zero + 1][j_zero]
            tmp_state[i_zero + 1][j_zero] = 0

            # Only add if the parent does not have the same expanded state
            if self.parent and check_states_equal(tmp_state, self.parent.state):
                states_to_return.append(None)
            else:
                states_to_return.append(tmp_state)

        # Move Left: Swap zero and the element on its left if possible
        if (j_zero != 0):
            tmp_state = copy.deepcopy(self.state)
            tmp_state[i_zero][j_zero] = tmp_state[i_zero][j_zero - 1]
            tmp_state[i_zero][j_zero - 1] = 0

            # Only add if the parent does not have the same expanded state
            if self.parent and check_states_equal(tmp_state, self.parent.state):
                states_to_return.append(None)
            else:
                states_to_return.append(tmp_state)
        # ADD MORE LEGAL MOVES HERE
        return states_to_return




def tree_search(start_state, goal_state, heuristic, verbose="y"):
    root = Node(state=start_state)
    h = [] # Used as a heapq
    hq.heappush(h, root)
    explored = []
    max_nodes = 1
    expanded = 0

    # While heapq is not empty...
    while h:
        max_nodes = max(len(h), max_nodes)

        current = hq.heappop(h)

        if (check_states_equal(current.state, goal_state)):
            if (verbose == "y"):

                count=current.traceback()
                print()
                print('Move '+str(count+1))
                print()
                print_mystate(goal_state)
                print()
                print('Moves to solve misplaced tiles heuristic: '+str(count+1) + "\n")
                print("Expanded nodes to solve using misplaced tiles heuristic: " + str(expanded) + " nodes.")


            return expanded, max_nodes,current.t_count()

        else:
            explored.append(current)

            # Filter out None states
            expanded_states_list = [non_none_state for non_none_state in current.expand() if non_none_state]

            # If expanded_states_list is empty after we filter all None, continue
            if expanded_states_list == []:
                continue

            for expanded_state in expanded_states_list:
                new_node = Node(state=expanded_state)

                # If h is not empty and new_node already exists in h, continue
                # If explored is not empty and new_node already exists in explored, continue
                if ((h and new_node in h) or (explored and new_node in explored)):
                    # already seen
                    continue

                # Depending on the heuristic, calculate costs
                if (heuristic == 1):
                    new_node.cost_to_goal = calc_misplaced_tiles(new_node.state, goal_state)

                if (heuristic == 2):
                    new_node.cost_to_goal = calc_manhattan_dist(new_node.state, goal_state)

                # if (heuristic == "XYZ") ADD MORE HEURISTIC CHOICES HERE

                current.add_child(node=new_node)

                # Referenced https://docs.python.org/3/library/heapq.html#priority-queue-implementation-notes
                # NOTE: F-VAL = (g(n) + h(n)) IS AUTOMATICALLY CALCULATED WHEN HEAPQ SORTS NODES
                # SEE def __lt__(self, other) IN THE NODE CLASS
                hq.heappush(h, new_node)

            expanded += 1

    print("Couldn't find a solution :(")
    return -1



In [2]:
init_state = [[1, 2, 3], [5, 6, 0], [7, 8, 4]]
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

alg = 1 # A* Misplaced Tile Heuristic 3 manhaten
verbose ='n'
print('Initial State:\n')
print_mystate(init_state)
print('\n\n')
print('Goal State:\n')
print_mystate(goal_state)
expanded, max_nodes,tt=tree_search(init_state, goal_state, alg, verbose)
print()
print('Moves to solve misplaced tiles heuristic: ',tt)
print('Expanded nodes to solve using misplaced tiles heuristic: ',expanded)
alg = 2 # A* Misplaced Tile Heuristic 3 manhaten
verbose ='n'
expanded, max_nodes,tt=tree_search(init_state, goal_state, alg, verbose)
print('Moves to solve using Manhattan distance heuristic: ',tt)
print('Expanded nodes to solve using Manhattan distance heuristic: ',expanded)


Initial State:

1 2 3
5 6 0
7 8 4



Goal State:

1 2 3
4 5 6
7 8 0

Moves to solve misplaced tiles heuristic:  13
Expanded nodes to solve using misplaced tiles heuristic:  200
Moves to solve using Manhattan distance heuristic:  13
Expanded nodes to solve using Manhattan distance heuristic:  82


In [3]:
alg = 1
verbose='y'
expanded, max_nodes,tt=tree_search(init_state, goal_state, alg, verbose)

Moves to solve:
1 2 3
4 5 6
7 8 0

Move  0

1 2 3
4 5 0
7 8 6

Move  1

1 2 3
4 0 5
7 8 6

Move  2

1 2 3
0 4 5
7 8 6

Move  3

1 2 3
7 4 5
0 8 6

Move  4

1 2 3
7 4 5
8 0 6

Move  5

1 2 3
7 0 5
8 4 6

Move  6

1 2 3
7 5 0
8 4 6

Move  7

1 2 3
7 5 6
8 4 0

Move  8

1 2 3
7 5 6
8 0 4

Move  9

1 2 3
7 5 6
0 8 4

Move  10

1 2 3
0 5 6
7 8 4

Move  11

1 2 3
5 0 6
7 8 4

Move  12

1 2 3
5 6 0
7 8 4

Move 13

1 2 3
4 5 6
7 8 0

Moves to solve misplaced tiles heuristic: 13

Expanded nodes to solve using misplaced tiles heuristic: 200 nodes.


In [4]:

import numpy as np
import pandas as pd
import random

# Creating Empty DataFrame and Storing it in variable df
df = pd.DataFrame()
df = pd.DataFrame(columns=['d', 'EBF misplaced Tiles', 'EBF Manhattan Distance'])


In [10]:
puzzle =[[[0, 1, 2], [4, 5, 3], [7,8, 6]],[[0, 1, 3], [4, 2, 5], [7, 8, 6]],
        [[1, 8, 2], [0, 4, 3], [7, 6, 5]],[[1, 0, 2], [6, 3, 4], [7,5, 8]],
         [[1, 2, 3], [5, 6, 0], [7, 8, 4]],[[3, 0, 7], [2, 8, 1], [6, 4, 5]],
          [[2,8, 0], [1, 5, 3], [6,4, 7]],[[7,2, 4], [5, 0, 6], [8,3,1]],
           [[1,0, 4], [2, 5, 3], [8,7,6]],[[2,8, 4], [7, 6,1], [3,5,0]],
        [[8, 7, 4], [3, 2, 6] ,[0, 5, 1]],[[8, 7, 4], [0, 2, 6] ,[3, 5, 1]],
         [[8, 7, 4], [2, 0, 6] ,[3, 5, 1]]]
for i in range(len(puzzle)):
    print(i)
    #x=random.sample(range(9),9)
    #y=np.reshape(x,(3,3))
    #puzzle =np.ndarray.tolist(y)
    # using equation N=p+p^2+p^3+...+p^d
    #j=0
    #if(isSolvable(puzzle)) :
    alg=1
    verbose = 'n'
    print('step 1')
    init_state = puzzle[i]
    goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
    expanded, max_nodes,tt=tree_search(init_state, goal_state, alg, verbose)
    coeffs = np.ones((1,tt))
    coeffs =np.append(coeffs,[-expanded])
    r=np.roots(coeffs)
    xx=max(r[np.isreal(r)].real)
    alg=2
    print('step 2')
    expanded, max_nodes,tt=tree_search(init_state, goal_state, alg, verbose)

    coeffs = np.ones((1,tt))
    coeffs =np.append(coeffs,[-expanded])
    r=np.roots(coeffs)
    yy=max(r[np.isreal(r)].real)
    df.loc[i]=[tt,xx,yy]
    #j+=1

step 2
6
step 1
step 2
7
step 1
step 2
8
step 1
step 2
9
step 1
step 2
10
step 1
step 2
11
step 1
step 2
12
step 1
step 2


In [11]:
df

,d,EBF misplaced Tiles,EBF Manhattan Distance
0,4.0,1.000000,1.000000
1,4.0,1.000000,1.000000
2,9.0,1.229243,1.039890
3,15.0,1.360961,1.226147
4,13.0,1.358755,1.243973
5,23.0,1.419156,1.224989
6,18.0,1.375945,1.200148
7,22.0,1.410768,1.247221
8,21.0,1.430415,1.310771
9,24.0,1.412893,1.229788
